In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime

In [3]:
# Read csv file, convert time to datetime data type
df = pd.read_csv('flex_networks.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'], infer_datetime_format=True)

In [6]:
peaks = []
loads = df['crawfordCrescent_F2'].values

n = 48
dayLoads = [loads[i:i + n] for i in range(0, len(loads), n)]

for day in dayLoads:
    peaks.append(max(day))

[array([ 7.34716667,  6.4978    ,  6.62123333,  5.8143    ,  4.5665    ,
        4.94946667,  5.09793333,  4.54216667,  5.26833333,  4.625     ,
        5.40406667,  4.69236667,  4.83353333,  5.50996667,  5.6439    ,
        5.9392    ,  7.84533333, 11.98923333, 12.22776667, 12.7884    ,
       13.20473333, 15.512     , 18.72563333, 15.15936667, 12.40393333,
       13.86366667, 12.4227    , 14.57016667, 13.6908    , 10.59843333,
       14.55483333, 14.96003333, 17.54796667, 19.82933333, 20.73733333,
       18.56256667, 14.02523333, 10.6895    , 12.4649    , 11.7389    ,
       10.959     , 11.38343333, 10.64803333, 10.90586667, 12.00376667,
        9.98866667, 10.17843333,  8.05706667]), array([ 7.84923333,  7.1452    ,  6.5875    ,  5.0575    ,  4.93393333,
        4.78343333,  4.6821    ,  4.1574    ,  4.6002    ,  4.77853333,
        4.85886667,  4.6898    ,  4.69136667,  5.27926667,  6.94116667,
        7.78066667, 11.42593333,  9.05806667,  9.1261    , 11.09376667,
       10.5606 